<a href="https://colab.research.google.com/github/baeseungyou/study/blob/main/9%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 필요한 라이브러리 임포트
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import requests

# 데이터 로드
korean_sentences = []
french_sentences = []

if not os.path.exists('kor.txt') or not os.path.exists('fra.txt'):
    print("데이터 파일을 다운로드합니다...")
    kor_url = "https://raw.githubusercontent.com/yourusername/yourrepo/main/kor.txt"  # 실제 URL로 변경
    fra_url = "https://raw.githubusercontent.com/yourusername/yourrepo/main/fra.txt"  # 실제 URL로 변경

    # 파일을 다운로드하고 저장합니다.
    try:
        with open('kor.txt', 'w', encoding='utf-8') as ko_file:
            ko_file.write(requests.get(kor_url).text)
        with open('fra.txt', 'w', encoding='utf-8') as fr_file:
            fr_file.write(requests.get(fra_url).text)
        print("데이터 파일 다운로드가 완료되었습니다.")
    except requests.exceptions.RequestException as e:
        print(f"다운로드 중 오류가 발생했습니다: {e}")
else:
    print("데이터 파일이 이미 존재합니다.")

# 파일을 읽어오기
with open('kor.txt', 'r', encoding='utf-8') as ko_file:
    korean_sentences = [line.strip() for line in ko_file.readlines()]

with open('fra.txt', 'r', encoding='utf-8') as fr_file:
    french_sentences = [line.strip() for line in fr_file.readlines()]

# 데이터 전처리: 텍스트 토큰화
tokenizer_ko = keras.preprocessing.text.Tokenizer()
tokenizer_fr = keras.preprocessing.text.Tokenizer()

tokenizer_ko.fit_on_texts(korean_sentences)
tokenizer_fr.fit_on_texts(french_sentences)

# 단어 집합 크기
vocab_size_ko = len(tokenizer_ko.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1

# 시퀀스 패딩
max_length_ko = max(len(s.split()) for s in korean_sentences)
max_length_fr = max(len(s.split()) for s in french_sentences)

X = tokenizer_ko.texts_to_sequences(korean_sentences)
y = tokenizer_fr.texts_to_sequences(french_sentences)

# 시퀀스 길이에 맞춰 패딩
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=max_length_ko)
y = keras.preprocessing.sequence.pad_sequences(y, maxlen=max_length_fr)

# 디코더 입력 데이터 생성
y_input = y[:, :-1]
y_output = y[:, 1:]

# 모델 정의
model = keras.Sequential()
model.add(layers.Embedding(vocab_size_ko, 256, input_length=max_length_ko))
model.add(layers.LSTM(256))
model.add(layers.RepeatVector(max_length_fr - 1))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.Dense(vocab_size_fr, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X, np.expand_dims(y_input, -1), epochs=50, batch_size=64)

# 번역 함수 정의
def translate(sentence):
    seq = tokenizer_ko.texts_to_sequences([sentence])
    padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length_ko)

    predictions = model.predict(padded)
    predicted_indices = np.argmax(predictions, axis=-1)

    # 인덱스를 단어로 변환
    translated_sentence = ' '.join(tokenizer_fr.index_word[i] for i in predicted_indices[0] if i != 0)
    return translated_sentence

# 번역 및 결과 출력
print(f"한국어 문장 수: {len(korean_sentences)}")
print(f"프랑스어 문장 수: {len(french_sentences)}")

num_sentences = min(5, len(korean_sentences))  # 최대 5개 문장 또는 리스트 크기
for i in range(num_sentences):
    input_sentence = korean_sentences[i]
    correct_translation = french_sentences[i]
    translated_sentence = translate(input_sentence)

    print(f"입력 문장: {input_sentence}")
    print(f"정답 문장: {correct_translation}")
    print(f"번역 문장: {translated_sentence}\n")

데이터 파일이 이미 존재합니다.
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 1.3861
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5000 - loss: 1.3710
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 1.3538
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.3320
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.3028
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5000 - loss: 1.2639
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5000 - loss: 1.2126
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.1466
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.0643
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 0.9668
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 0.8614
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5000 

In [2]:
# 필요한 라이브러리 임포트
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import requests
import pandas as pd
from nltk.tokenize import word_tokenize

# 데이터 로드
korean_sentences = []
french_sentences = []

if not os.path.exists('kor.txt') or not os.path.exists('fra.txt'):
    print("데이터 파일을 다운로드합니다...")
    kor_url = "https://raw.githubusercontent.com/yourusername/yourrepo/main/kor.txt"  # 실제 URL로 변경
    fra_url = "https://raw.githubusercontent.com/yourusername/yourrepo/main/fra.txt"  # 실제 URL로 변경

    # 파일을 다운로드하고 저장합니다.
    try:
        with open('kor.txt', 'w', encoding='utf-8') as ko_file:
            ko_file.write(requests.get(kor_url).text)
        with open('fra.txt', 'w', encoding='utf-8') as fr_file:
            fr_file.write(requests.get(fra_url).text)
        print("데이터 파일 다운로드가 완료되었습니다.")
    except requests.exceptions.RequestException as e:
        print(f"다운로드 중 오류가 발생했습니다: {e}")
else:
    print("데이터 파일이 이미 존재합니다.")

# 파일을 읽어오기
korean_data = pd.read_csv('kor.txt', header=None)
french_data = pd.read_csv('fra.txt', header=None)

# 데이터가 없는 경우 오류 처리
if korean_data.empty or french_data.empty:
    raise ValueError("데이터 파일이 비어 있습니다. 파일을 확인해주세요.")

korean_sentences = korean_data[0].tolist()
french_sentences = french_data[0].tolist()

# 데이터 전처리: 텍스트 토큰화
tokenizer_ko = keras.preprocessing.text.Tokenizer()
tokenizer_fr = keras.preprocessing.text.Tokenizer()

tokenizer_ko.fit_on_texts(korean_sentences)
tokenizer_fr.fit_on_texts(french_sentences)

# 단어 집합 크기
vocab_size_ko = len(tokenizer_ko.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1

# 시퀀스 패딩
max_length_ko = max(len(s.split()) for s in korean_sentences)
max_length_fr = max(len(s.split()) for s in french_sentences)

X = tokenizer_ko.texts_to_sequences(korean_sentences)
y = tokenizer_fr.texts_to_sequences(french_sentences)

# 시퀀스 길이에 맞춰 패딩
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=max_length_ko)
y = keras.preprocessing.sequence.pad_sequences(y, maxlen=max_length_fr)

# 디코더 입력 데이터 생성
y_input = y[:, :-1]
y_output = y[:, 1:]

# 모델 정의
model = keras.Sequential()
model.add(layers.Embedding(vocab_size_ko, 256, input_length=max_length_ko))
model.add(layers.LSTM(256))
model.add(layers.RepeatVector(max_length_fr - 1))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.Dense(vocab_size_fr, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X, np.expand_dims(y_input, -1), epochs=50, batch_size=64)

# 번역 함수 정의
def translate(sentence):
    # 입력 문장을 시퀀스로 변환
    seq = tokenizer_ko.texts_to_sequences([sentence])
    padded_seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length_ko)

    # 예측
    pred = model.predict(padded_seq)
    pred_idx = np.argmax(pred, axis=-1)

    # 인덱스를 단어로 변환
    translated_sentence = tokenizer_fr.sequences_to_texts(pred_idx)[0]
    return translated_sentence

# 테스트할 문장
test_sentence = "안녕하세요"
predicted_translation = translate(test_sentence)

# 결과 출력
print("입력 문장:", test_sentence)
print("정답 문장:", "안녕하세요")  # 정답 문장은 실제 정답으로 교체
print("번역 문장:", predicted_translation)

데이터 파일이 이미 존재합니다.
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.3887
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5000 - loss: 1.3728
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.3554
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5000 - loss: 1.3338
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 1.3054
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5000 - loss: 1.2677
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 1.2181
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5000 - loss: 1.1542
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 1.0747
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.9808
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5000 - loss: 0.8793
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 0.7856
Epoch 13/5